# **Data Formatting Pipeline**

#### Llegir dades i paquets necessaris:

In [ ]:
import pyspark
from pyspark.sql import SparkSession
import duckdb

!wget -O "duckdb.jar" "https://repo1.maven.org/maven2/org/duckdb/duckdb_jdbc/0.10.1/duckdb_jdbc-0.10.1.jar"

spark = SparkSession.builder\
    .appName("Parquet to DuckDB") \
    .master("local[*]") \
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

# Leer el archivo Parquet
demography_df = spark.read.parquet('../data/demography.parquet')
mh_df = spark.read.parquet('../data/mental_health.parquet')
society_df = spark.read.parquet('../data/society.parquet')

### Save Basic

In [2]:
d = {
    demography_df:"demography_df",
    mh_df:"mh_df",
    society_df:'society_df'
}

for k,v in d.items():
    k.write \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:{v}.db") \
        .option("dbtable", f"{v}") \
        .option("driver", "org.duckdb.DuckDBDriver") \
        .save()

### Read Bàsic

In [ ]:
DEMOGRAPHY = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:demography_df.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("dbtable","demography_df") \
  .load()

MH = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:mh_df.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("dbtable","mh_df") \
  .load()

SOCIETY = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:society_df.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("dbtable","society_df") \
  .load()